In [2]:
import os
import umps


In [1]:
import torchvision
import torchvision.transforms as transforms
import torch
import torch.utils.data

def embedding_pixel(batch, label: int = 0):
    pixel_size = batch.shape[-1] * batch.shape[-2]
    x = batch.view(*batch.shape[:-2], pixel_size)
    # x[:] = 0
    x = torch.stack([x, 1-x], dim=-1)
    # x = x / torch.sum(x, dim=-1).unsqueeze(-1)
    x = x / torch.norm(x, dim=-1).unsqueeze(-1)
    return x

def embedding_label(labels: torch.Tensor):
    emb = torch.zeros(labels.shape[0], 2)
    emb[torch.arange(labels.shape[0]), labels] = 1
    return emb

def filiter_single_channel(batch):
    return batch[0, ...]

def filter_dataset(dataset, allowed_digits=[0, 1]):
    indices = []
    for i in range(len(dataset)):
        _, label = dataset[i]
        if label in allowed_digits:
            indices.append(i)
    return torch.utils.data.Subset(dataset, indices)

img_size = 16
transform = transforms.Compose([
    transforms.Resize(img_size),
    transforms.ToTensor(),
    transforms.Lambda(filiter_single_channel),
    transforms.Lambda(embedding_pixel),
])

trainset = torchvision.datasets.QMNIST(
    root="data",
    train=True,
    download=True,
    transform=transform
)

trainset = filter_dataset(trainset, allowed_digits=[0, 1])

trainloader = torch.utils.data.DataLoader(
    trainset,
    batch_size=128,
    shuffle=True,
)


In [3]:
import unitary_optimizer
umpsm = umps.uMPS(N = 16 * 16, chi = 2, d = 2, l = 2, layers = 1, device = "cpu")


Path is not set, setting...
Found the path
Initialized MPS unitaries


In [4]:
def loss_batch(outputs, labels):
    device = outputs.device
    loss = torch.zeros(1, device=device, dtype=torch.float64)

    for i in range(len(outputs)):
        prob = outputs[i] if labels[i] == 0 else 1 - outputs[i]
        loss -= torch.log(prob + 1e-8)
    return loss

def calculate_accuracy(outputs, labels):
    predictions = (outputs < 0.5).float()
    correct = (predictions == labels).float().sum()
    accuracy = correct / labels.numel()
    return accuracy.item()

In [26]:
u = umpsm.params[-1].grad.reshape(4, 4)
u @ u.T

AttributeError: 'NoneType' object has no attribute 'reshape'

In [28]:
umpsm_op = unitary_optimizer.Adam(umpsm, lr=0.01)

for epoch in range(100):
    acc = 0
    for data, target in trainloader:    
        data = data.permute(1, 0, 2)
        umpsm_op.zero_grad()
        outputs = umpsm(data)
        loss = loss_batch(outputs, target)
        loss.backward()
        umpsm_op.step()

        # Calculate accuracy
        accuracy = calculate_accuracy(outputs, target)
        # print(f"Accuracy: {accuracy:.4f}")
        acc += accuracy
    acc /= len(trainloader)
    print(f"Accuracy: {acc:.4f}")
    # print("grad", umpsm.params[-1].grad)







KeyboardInterrupt: 

In [5]:
data, target = next(iter(trainloader))

In [12]:
mps.kraus_ops

torch.Size([2, 4, 4])

In [15]:
I = torch.zeros(4, 4)

for kraus_op in mps.kraus_ops[0]:
    I += kraus_op.T @ kraus_op

I


tensor([[ 1.0000e+00, -1.5151e-09,  3.6242e-10, -8.2355e-09],
        [-1.5151e-09,  1.0000e+00,  3.9718e-09,  5.4807e-09],
        [ 3.6242e-10,  3.9718e-09,  1.0000e+00,  7.3616e-09],
        [-8.2355e-09,  5.4807e-09,  7.3616e-09,  1.0000e+00]],
       grad_fn=<AddBackward0>)

In [24]:
rho[0]

tensor([[0.4929, 0.1871],
        [0.1871, 0.5071]], dtype=torch.float64, grad_fn=<SelectBackward0>)

In [84]:
import tpcp_mps
from importlib import reload
reload(tpcp_mps)

mps = tpcp_mps.MPSTPCP(N=16 * 16, K=1, d=2)

outputs = mps.forward(data[:, :, :].to(torch.float64))

In [85]:
rand_gen = torch.randn(100, 16 * 16, 2)
rand_gen /= torch.norm(rand_gen, dim=-1).unsqueeze(-1)

mps.forward(rand_gen.to(torch.float64))

tensor([0.3217, 0.5300, 0.6689, 0.4137, 0.4518, 0.4848, 0.4619, 0.6848, 0.8434,
        0.4206, 0.6056, 0.4578, 0.6694, 0.7811, 0.2808, 0.4834, 0.4791, 0.4612,
        0.0448, 0.7090, 0.4482, 0.6645, 0.3723, 0.4935, 0.3612, 0.5337, 0.6680,
        0.3314, 0.4152, 0.8303, 0.4398, 0.4487, 0.7049, 0.9043, 0.7533, 0.2612,
        0.8024, 0.4165, 0.2396, 0.3580, 0.3231, 0.2031, 0.5297, 0.4053, 0.1495,
        0.6114, 0.3828, 0.1742, 0.3090, 0.6286, 0.4317, 0.6117, 0.6485, 0.6831,
        0.4552, 0.5844, 0.5443, 0.6155, 0.3997, 0.4054, 0.1457, 0.1112, 0.3017,
        0.4169, 0.4252, 0.3610, 0.7113, 0.3329, 0.3020, 0.2280, 0.4437, 0.6049,
        0.2926, 0.2494, 0.5580, 0.5982, 0.6938, 0.4332, 0.3964, 0.6479, 0.6917,
        0.5261, 0.6665, 0.5378, 0.5206, 0.6813, 0.5700, 0.5287, 0.3308, 0.7020,
        0.4728, 0.6162, 0.4287, 0.5274, 0.7051, 0.4734, 0.4715, 0.1893, 0.6181,
        0.4499], dtype=torch.float64, grad_fn=<SelectBackward0>)

In [132]:
B.shape

torch.Size([2, 1, 4, 4])

In [157]:
G = mps.kraus_ops[-1].grad[0]
G = G / torch.norm(G)
K = mps.kraus_ops[-1].detach()[0]


A = torch.cat([G, K], dim=1)
B = torch.cat([K, -G], dim=1)

rg = A @ torch.linalg.inv(torch.eye(8) + 0.5 * 0.05 * B.T @ A) @ B.T @ K
Kp = K - rg * 0.05

Kp @ Kp.T

tensor([[ 1.0000e+00,  5.5511e-17, -5.5511e-17,  0.0000e+00],
        [ 5.5511e-17,  1.0000e+00, -2.7756e-17,  2.7756e-17],
        [-5.5511e-17, -2.7756e-17,  1.0000e+00,  5.5511e-17],
        [ 0.0000e+00,  2.7756e-17,  5.5511e-17,  1.0000e+00]],
       dtype=torch.float64)

In [165]:
mps.kraus_ops[0].grad

tensor([[[ 0.0000,  0.0000,  0.0000,  1.9005],
         [ 0.0000,  0.0000,  0.0000, -0.1128],
         [ 0.0000,  0.0000,  0.0000, 21.8310],
         [ 0.0000,  0.0000,  0.0000,  2.1302]]], dtype=torch.float64)

In [176]:
optimizer.params[0]

Parameter containing:
tensor([[[-0.0781, -0.2654,  0.9404, -0.1976],
         [ 0.5015, -0.7846, -0.2378, -0.2761],
         [-0.4079,  0.1087, -0.1895, -0.8865],
         [-0.7589, -0.5496, -0.1522,  0.3143]]], dtype=torch.float64,
       requires_grad=True)

In [191]:
optimizer.params[0][0] @ optimizer.params[0][0].T

tensor([[ 1.0000e+00, -3.3307e-16, -2.6368e-16,  4.1633e-17],
        [-3.3307e-16,  1.0000e+00,  2.2898e-16, -3.6082e-16],
        [-2.6368e-16,  2.2898e-16,  1.0000e+00, -2.2204e-16],
        [ 4.1633e-17, -3.6082e-16, -2.2204e-16,  1.0000e+00]],
       dtype=torch.float64, grad_fn=<MmBackward0>)

In [189]:
mps.kraus_ops[0]

Parameter containing:
tensor([[[-0.0462, -0.5530, -0.5585, -0.6166],
         [ 0.3898, -0.3660,  0.7532, -0.3832],
         [ 0.6462, -0.4353, -0.2547,  0.5727],
         [-0.6544, -0.6089,  0.2366,  0.3808]]], dtype=torch.float64,
       requires_grad=True)

In [201]:
K = mps.kraus_ops[0].reshape(8, 4) 

K.T @ K


tensor([[ 1.0000e+00, -1.6662e-16, -6.0210e-17, -4.3451e-17],
        [-1.6662e-16,  1.0000e+00,  1.8244e-17, -6.5638e-17],
        [-6.0210e-17,  1.8244e-17,  1.0000e+00,  1.2042e-16],
        [-4.3451e-17, -6.5638e-17,  1.2042e-16,  1.0000e+00]],
       dtype=torch.float64, grad_fn=<MmBackward0>)

In [221]:
mps.kraus_ops[0].T @ mps.kraus_ops[0]

tensor([[ 1.0000e+00, -9.8817e-19,  1.8542e-17,  1.5244e-16],
        [-9.8817e-19,  1.0000e+00, -2.7013e-18, -3.7641e-17],
        [ 1.8542e-17, -2.7013e-18,  1.0000e+00, -1.6334e-17],
        [ 1.5244e-16, -3.7641e-17, -1.6334e-17,  1.0000e+00]],
       dtype=torch.float64, grad_fn=<MmBackward0>)

In [230]:
import kraus_optimizer
reload(kraus_optimizer)
reload(tpcp_mps)
# mps = tpcp_mps.MPSTPCP(N=16 * 16, K=2, d=2)
# optimizer = kraus_optimizer.Adam(mps.kraus_ops, lr=0.0001)
optimizer = kraus_optimizer.CayleySGDMomentum(mps.kraus_ops, lr=0.0001, beta=0.95, q=0.5, s=4)

for _ in range(1000):
    optimizer.zero_grad()
    outputs = mps.forward(data[:, :, :].to(torch.float64))
    loss = loss_batch(outputs, target)
    loss.backward()
    optimizer.step()
    print(loss.item())
    print(calculate_accuracy(outputs, target))

# for epoch in range(100):
#     acc_tot = 0
#     loss_tot = 0
#     for data, target in trainloader:
#         optimizer.zero_grad()
#         outputs = mps.forward(data[:, :, :].to(torch.float64))
#         loss = loss_batch(outputs, target)
#         loss.backward()
#         optimizer.step()

#         acc = calculate_accuracy(outputs, target)
#         acc_tot += acc
#         loss_tot += loss.item()
#     acc_tot /= len(trainloader)
#     loss_tot /= len(trainloader)
#     print(f"Accuracy: {acc_tot:.4f}")
#     print(f"Loss: {loss_tot:.4f}")


88.33191124933472
0.5390625
88.33190862632068
0.5390625
88.33190369198766
0.5390625
88.33189689400702
0.5390625
88.3318887714528
0.5390625
88.33187990826016
0.5390625
88.331870888128
0.5390625
88.3318622538474
0.5390625
88.33185447344013
0.5390625
88.3318479147869
0.5390625
88.33184282967609
0.5390625
88.33183934745813
0.5390625
88.33183747780937
0.5390625
88.33183712152645
0.5390625
88.33183808782029
0.5390625
88.33184011627898
0.5390625
88.33184290152242
0.5390625
88.33184611857837
0.5390625
88.33184944715126
0.5390625
88.33185259321003
0.5390625
88.33185530665929
0.5390625
88.33185739425168
0.5390625
88.33185872730965
0.5390625
88.33185924422692
0.5390625
88.3318589480855
0.5390625
88.3318579000275
0.5390625
88.33185620925777
0.5390625
88.3318540206942
0.5390625
88.33185150135407
0.5390625
88.33184882654099
0.5390625
88.33184616680894
0.5390625
88.33184367653107
0.5390625
88.33184148470836
0.5390625
88.3318396884362
0.5390625
88.33183834922204
0.5390625
88.33183749213111
0.5390625
8

KeyboardInterrupt: 

In [236]:
mps.kraus_ops[-1].gra

tensor([[     0.0000, -20810.3705,      0.0000,  19047.8718],
        [     0.0000,      0.0000,      0.0000,      0.0000],
        [     0.0000, -25382.7747,      0.0000,   7647.4490],
        [     0.0000,      0.0000,      0.0000,      0.0000]],
       dtype=torch.float64)

In [257]:
import kraus_optimizer

reload(kraus_optimizer)

# mps_tpcp = tpcp_mps.MPSTPCP(N=16 * 16, K=1, d=2)
# optimizer = kraus_optimizer.CayleySGDMomentum(mps.kraus_ops, lr=0.005, beta=0.5, q=0.5, s=2)
optimizer = kraus_optimizer.CayleyAdam(mps_tpcp.kraus_ops, lr=0.01)

for epoch in range(100):
    acc_tot = 0
    loss_tot = 0
    for data, target in trainloader:
        optimizer.zero_grad()
        outputs = mps_tpcp.forward(data.to(torch.float64))
        loss = loss_batch(outputs, target)
        loss.backward()
        optimizer.step()

        acc = calculate_accuracy(outputs, target)
        acc_tot += acc
        loss_tot += loss.item()
        # print(loss.item())
        # print(mps.kraus_ops[-1].sum())
    acc_tot /= len(trainloader)
    loss_tot /= len(trainloader)
    print(f"Accuracy: {acc_tot:.4f}")
    print(f"Loss: {loss_tot:.4f}")


Accuracy: 0.5021
Loss: 89.4934
Accuracy: 0.5324
Loss: 88.4154


KeyboardInterrupt: 

In [258]:
for i, kraus_op in enumerate(mps_tpcp.kraus_ops):
    kraus_op.data[:] = umpsm.params[i].reshape(4,4)
    print(mps_tpcp.forward(data.to(torch.float64)))

In [252]:
umpsm.params[0]

Parameter containing:
tensor([[[[ 0.5022, -0.2951],
          [ 0.5226,  0.6225]],

         [[-0.5633, -0.2183],
          [-0.4017,  0.6883]]],


        [[[ 0.4872, -0.5938],
          [-0.6219, -0.1525]],

         [[ 0.4394,  0.7160],
          [-0.4228,  0.3399]]]], dtype=torch.float64, requires_grad=True)

In [254]:
# umpsm = umps.uMPS(N = 16 * 16, chi = 2, d = 2, l = 2, layers = 1, device = "cpu")
umpsm_op = unitary_optimizer.Adam(umpsm, lr=0.01)
loss_list = []

for epoch in range(100):
    acc = 0
    for data, target in trainloader:    
        data = data.permute(1, 0, 2)
        umpsm_op.zero_grad()
        outputs = umpsm(data)
        loss = loss_batch(outputs, target)
        loss.backward()
        loss_list.append(loss.item())
        umpsm_op.step()

        # Calculate accuracy
        accuracy = calculate_accuracy(outputs, target)
        # print(f"Accuracy: {accuracy:.4f}")
        acc += accuracy
    acc /= len(trainloader)
    print(f"Accuracy: {acc:.4f}")
    print(f"loss: {loss.item()}")

Accuracy: 0.9207
loss: 32.39418858531672


KeyboardInterrupt: 